In [92]:
print('Starting NFL Fantasy Pipeline...')
print('Pipeline complete!')

Starting NFL Fantasy Pipeline...
Pipeline complete!


In [93]:
#=========================== EXTRACTION ===========================
# sample data

players = [
    {
        'name': 'Josh Allen'
        ,'position': 'QB'
        ,'passing_yards': 300
        ,'passing_touchdowns': 2
        ,'rushing_yards': 50
        ,'rushing_touchdowns': 1
        ,'receptions': 0
        ,'receiving_yards': 0
        ,'receiving_touchdowns': 0
    },
    {
        'name': 'Derrick Henry'
        ,'position': 'RB'
        ,'passing_yards': 0
        ,'passing_touchdowns': 0
        ,'rushing_yards': 120
        ,'rushing_touchdowns': 2
        ,'receptions': 3
        ,'receiving_yards': 25
        ,'receiving_touchdowns': 0
    },
    {
        'name': 'Davante Adams'
        ,'position': 'WR'
        ,'passing_yards': 0
        ,'passing_touchdowns': 0
        ,'rushing_yards': 0
        ,'rushing_touchdowns': 0
        ,'receptions': 8
        ,'receiving_yards': 95
        ,'receiving_touchdowns': 1
    }
]

# print(f'Our sample player: {player}')

In [94]:
#=========================== TRANSFORM ===========================
# calculate fantasy points

def calculate_fantasy_points(player): 
    # Passing yards: 1 point per 25 yards (so 300 yards = 12 points)
    # Passing touchdowns: 4 points each
    # Rushing yards: 1 point per 10 yards
    # Rushing touchdowns: 6 points each
    # Receptions: 1 point each
    # Receiving Yards: 1 point per 10 yards
    # Receiving TDs: 6 points each

    passing_points = player['passing_yards'] / 25 * 1
    passing_td_points = player['passing_touchdowns'] * 4
    rushing_points = player['rushing_yards'] / 10 * 1
    rushing_td_points = player['rushing_touchdowns'] * 6
    reception_points = player['receptions'] * 1
    receiving_points = player['receiving_yards'] / 10 * 1
    receiving_td_points = player['receiving_touchdowns'] * 6

    total_points = (passing_points 
                    + passing_td_points 
                    + rushing_points 
                    + rushing_td_points
                    + reception_points
                    + receiving_points 
                    + receiving_td_points
    )

    return total_points

print(f'We now have {len(players)} players')
print('Players:', [p['name'] for p in players])

We now have 3 players
Players: ['Josh Allen', 'Derrick Henry', 'Davante Adams']


In [95]:
# calculate for all players in dataset

# iterate through every player in the dataset to calculate their total fantasy points
for player in players:
    points = calculate_fantasy_points(player)
    print(f'{player['name']} ({player['position']}): {points} fantasy points')

Josh Allen (QB): 31.0 fantasy points
Derrick Henry (RB): 29.5 fantasy points
Davante Adams (WR): 23.5 fantasy points


In [96]:
#=========================== LOAD ===========================

import sqlite3

# connect to database
conn = sqlite3.connect('nfl_fantasy.db')
print('Connected to database!')
cursor = conn.cursor()

#helps when re-running the code
cursor.execute('''drop table if exists player_stats''')

# create table for player/stats
cursor.execute('''
               create table player_stats (
                name text
                ,position text
                ,week integer
                ,fantasy_points real
                ,primary key (name, week)
                )
''')

conn.commit()
conn.close()
print('Table created!')

Connected to database!
Table created!


In [97]:
# insert player data 
conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

current_week = 1

for player in players:
    points = calculate_fantasy_points(player)
    cursor.execute('''
                insert or replace into player_stats 
                   (name
                   ,position
                   ,week
                   ,fantasy_points)
                values 
                   (?
                   ,?
                   ,?
                   ,?)
                ''', (player['name'], player['position'], current_week, points))
    
conn.commit()
conn.close()
print(f'Week {current_week} data inserted!')

Week 1 data inserted!


In [98]:
# read sql table to see if data made it in the db

conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

# pull all data from table
cursor.execute('select * from player_stats')
results = cursor.fetchall()

conn.close()

print('Data in database:')
for i in results:
    print(i)

Data in database:
('Josh Allen', 'QB', 1, 31.0)
('Derrick Henry', 'RB', 1, 29.5)
('Davante Adams', 'WR', 1, 23.5)


In [99]:
###################### Adding week 2 data to test if system can handle multiple weeks for the same players

Data in database:
('Josh Allen', 'QB', 1, 31.0)
('Derrick Henry', 'RB', 1, 29.5)
('Davante Adams', 'WR', 1, 23.5)


In [100]:
#=========================== extract ===========================

# Week 2 data - Josh Allen has a bad game, others do well
week2_players = [
    {
        "name": "Josh Allen"
        ,"position": "QB"
        ,"passing_yards": 200  # Bad passing game
        ,"passing_touchdowns": 1
        ,"rushing_yards": 20   # Less rushing
        ,"rushing_touchdowns": 0
        ,"receptions": 0
        ,"receiving_yards": 0
        ,"receiving_touchdowns": 0
    },
    {
        "name": "Derrick Henry"
        ,"position": "RB"
        ,"passing_yards": 0
        ,"passing_touchdowns": 0
        ,"rushing_yards": 180  # Big game!
        ,"rushing_touchdowns": 3
        ,"receptions": 2
        ,"receiving_yards": 30
        ,"receiving_touchdowns": 0
    },
    {
        "name": "Davante Adams"
        ,"position": "WR"
        ,"passing_yards": 0
        ,"passing_touchdowns": 0
        ,"rushing_yards": 0
        ,"rushing_touchdowns": 0
        ,"receptions": 12  # Monster receiving game
        ,"receiving_yards": 150
        ,"receiving_touchdowns": 2
    }
]

print("Week 2 player data created!")

Week 2 player data created!


In [101]:
#=========================== transform ===========================

for player in week2_players:
    points = calculate_fantasy_points(player)
    print(f'{player['name']} ({player['position']}): {points} fantasy points')

Josh Allen (QB): 14.0 fantasy points
Derrick Henry (RB): 41.0 fantasy points
Davante Adams (WR): 39.0 fantasy points


In [102]:
#=========================== load ===========================

conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

# insert week 2 data
current_week = 2

for player in week2_players:
    points = calculate_fantasy_points(player)
    cursor.execute('''
                insert or replace into player_stats 
                   (name
                   ,position
                   ,week
                   ,fantasy_points)
                values 
                   (?
                   ,?
                   ,?
                   ,?)
                ''', (player['name'], player['position'], current_week, points))
    
conn.commit()
conn.close()
print(f'Week {current_week} data inserted!')

In [103]:
# read sql table to see if week 2 data made it in the db

conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

cursor.execute('''select * from player_stats''')
results = cursor.fetchall()

conn.close()

print('All data in db:')
for i in results:
    print(i)

Week 2 data inserted!


In [104]:
#=========================== Try pulling in real data ===========================

All data in db:
('Josh Allen', 'QB', 1, 31.0)
('Derrick Henry', 'RB', 1, 29.5)
('Davante Adams', 'WR', 1, 23.5)
('Josh Allen', 'QB', 2, 14.0)
('Derrick Henry', 'RB', 2, 41.0)
('Davante Adams', 'WR', 2, 39.0)


In [105]:
import kagglehub
import shutil
import os

# download fantasy-football-agent-dataset from kaggle

# url: https://www.kaggle.com/datasets/sidsugathan/fantasy-football-agent-dataset

try:
    path = kagglehub.dataset_download("sidsugathan/fantasy-football-agent-dataset")
    print("Path to dataset files:", path)
    print("Dataset downloaded successfully!")

    # copy downloaded files to current project folder
    project_data_path = './data/'
    os.makedirs(project_data_path, exist_ok=True)

    # copy all csv files to project folder
    for file in os.listdir(path):
        if file.endswith('.csv'):
            source = os.path.join(path, file)
            destination = os.path.join(project_data_path, file)
            shutil.copy2(source, destination)   # .copy2() overwrites files that already exist, avoids dupes
            print(f'Copied {file} to ./data/')

    print(f'\nFiles now in your project at: {project_data_path}')

except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("This might require Kaggle authentication")

Path to dataset files: C:\Users\kcmcs\.cache\kagglehub\datasets\sidsugathan\fantasy-football-agent-dataset\versions\16
Dataset downloaded successfully!
Copied nfl_coaching_staff_2021_2024.csv to ./data/
Copied performance_data_with_ids_cleaned_2021_2024.csv to ./data/
Copied player_id_mapping_master_cleaned.csv to ./data/
Copied pre_draft_rankings_with_2024_tiers.csv to ./data/
Copied rookie_metadata_with_blurbs_and_ids_cleaned_2021_2024.csv to ./data/
Copied weekly_fantasy_scores_2024_with_ids.csv to ./data/

Files now in your project at: ./data/


In [106]:
# what's in the weekly_fantasy_scores_2024_with_ids.csv
import pandas as pd
df = pd.read_csv('./data/weekly_fantasy_scores_2024_with_ids.csv')

print('Dataset shape:', df.shape)
print('\nColumns:')
print(df.columns.tolist())

# df.head()

Dataset shape: (4407, 7)

Columns:
['player_id', 'player_name', 'position', 'team', 'week', 'points', 'canonical_player_id']


In [107]:
# explore full dataset
print(f'Dataset shape: {df.shape}')
print(f'Unique positions: {df['position'].unique()}')   #unique positions
print(f'Number of Unique Players: {df['player_name'].nunique()}')   #numb of unique players

# top 5 performances
print(f'\nTop Performances in dataset:')
top_weeks = df.nlargest(5, 'points')[['player_name', 'position', 'team', 'week', 'points']]
print(top_weeks.to_string())

# look at QBs to see if we can use this dataset to replace intial sample data above
print('\nSample QB data:')
qb_sample = df[df['position'] == 'QB'].head(3)
print(qb_sample[['player_name', 'team', 'week', 'points']].to_string)

Dataset shape: (4407, 7)
Unique positions: ['RB' 'WR' 'TE' 'QB' 'DST']
Number of Unique Players: 553

Top Performances in dataset:
         player_name position team  week  points
560   Ja'Marr Chase*       WR  CIN  10.0   55.40
139       Josh Allen       QB  BUF  14.0   51.88
1862  Jauan Jennings       WR   SF   3.0   46.50
227   Saquon Barkley       RB  NYG  12.0   46.20
563   Ja'Marr Chase*       WR  CIN  14.0   45.10

Sample QB data:
<bound method DataFrame.to_string of        player_name team  week  points
113  Brandon Allen  CIN  12.0    8.46
127     Josh Allen  BUF   1.0   31.18
128     Josh Allen  BUF   2.0    9.76>


In [ ]:
# begin to replace mock data with real data from week 1 - 20204 kaggle dataset
print('Getting real NFL players from 2024 Week 1')

# print(f'Week column dt: {df['week'].dtype}')      # double check 'week' dt

# pull josh allen's actual week 1 performance
josh_real = df[(df['player_name'] == 'Josh Allen') & (df['week'] == 1.0)]
print(josh_real)


# pull top RB from week 1
print('\nTop RB in Week 1:')
week1_rbs = df[(df['position'] == 'RB') & (df['week'] == 1.0)]
# print(week1_rbs)  #week 1 RBs
top_rb_week1 = week1_rbs.nlargest(1, 'points')
print(top_rb_week1)

# pull top WR from week 1
print('\nTop WR from Week 1:')
week1_wrs = df[(df['position'] == 'WR') & (df['week'] == 1.0)]
# print(week1_wrs)
top_wr_week1 = week1_wrs.nlargest(1, 'points')
print(top_wr_week1)

# pull top TE from week 1
print('\nTop TE from Week 1:')
week1_tes = df[(df['position'] == 'TE') & (df['week'] == 1.0)]
# print(week1_tes)
top_te_week1 = week1_tes.nlargest(1, 'points')
print(top_te_week1)

Getting real NFL players from 2024 Week 1
     player_id player_name position team  week  points canonical_player_id
127  alljos001  Josh Allen       QB  BUF   1.0   31.18           alljos001

Top RB in Week 1:
     player_id     player_name position team  week  points canonical_player_id
217  barsaq001  Saquon Barkley       RB  NYG   1.0    33.2           barsaq001

Top WR from Week 1:
      player_id  player_name position team  week  points canonical_player_id
3046  reejay001  Jayden Reed       WR   GB   1.0    33.1           reejay001

Top TE from Week 1:
      player_id    player_name position team  week  points canonical_player_id
2217  likisa001  Isaiah Likely       TE  BAL   1.0    26.1           likisa001


In [ ]:
# identify top performers in dataset
print("\nReal NFL Week 1 performers:")

if not josh_real.empty:
    player = josh_real.iloc[0]
    print(f"QB: {player['player_name']} ({player['team']}) - {player['points']} points")

if not top_rb_week1.empty:
    player = top_rb_week1.iloc[0]
    print(f"RB: {player['player_name']} ({player['team']}) - {player['points']} points")

if not top_wr_week1.empty:
    player = top_wr_week1.iloc[0]
    print(f"WR: {player['player_name']} ({player['team']}) - {player['points']} points")

if not top_te_week1.empty:
    player = top_te_week1.iloc[0]
    print(f"TE: {player['player_name']} ({player['team']}) - {player['points']} points")


Real NFL Week 1 performers:
QB: Josh Allen (BUF) - 31.18 points
RB: Saquon Barkley (NYG) - 33.2 points
WR: Jayden Reed (GB) - 33.1 points
TE: Isaiah Likely (BAL) - 26.1 points


In [124]:
# extract real NFL players from Week 1 data to replace mock data in our ETL pipeline

# create real_players list using the actual NFL data
real_players_week1 = []

if not josh_real.empty:
    real_players_week1.append({
        'name': josh_real.iloc[0]['player_name']
        ,'position': josh_real.iloc[0]['position']
        ,'team': josh_real.iloc[0]['team']
        ,'week': josh_real.iloc[0]['week']
        ,'fantasy_points': josh_real.iloc[0]['points']
    })

if not top_rb_week1.empty:
    real_players_week1.append({
        'name': top_rb_week1.iloc[0]['player_name']
        ,'position': top_rb_week1.iloc[0]['position']
        ,'team': top_rb_week1.iloc[0]['team']
        ,'week': top_rb_week1.iloc[0]['week']
        ,'fantasy_points': top_rb_week1.iloc[0]['points']
    })

if not top_wr_week1.empty:
    real_players_week1.append({
        'name': top_wr_week1.iloc[0]['player_name']
        ,'position': top_wr_week1.iloc[0]['position']
        ,'team': top_wr_week1.iloc[0]['team']
        ,'week': top_wr_week1.iloc[0]['week']
        ,'fantasy_points': top_wr_week1.iloc[0]['points']
    })

if not top_te_week1.empty:
    real_players_week1.append({
        'name': top_te_week1.iloc[0]['player_name']
        ,'position': top_te_week1.iloc[0]['position']
        ,'team': top_te_week1.iloc[0]['team']
        ,'week': top_te_week1.iloc[0]['week']
        ,'fantasy_points': top_te_week1.iloc[0]['points']
    })

print('Real players ready for ETL pipeline:')
for player in real_players_week1:
    print(f'{player['name']} ({player['position']}) - {player['fantasy_points']} points')


Real players ready for ETL pipeline:
Josh Allen (QB) - 31.18 points
Saquon Barkley (RB) - 33.2 points
Jayden Reed (WR) - 33.1 points
Isaiah Likely (TE) - 26.1 points


In [125]:
# Check current database contents
import sqlite3

conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM player_stats ORDER BY week, fantasy_points DESC')
results = cursor.fetchall()

conn.close()

print('Current data in database:')
for row in results:
    print(row)

Current data in database:
('Josh Allen', 'QB', 1, 31.0)
('Derrick Henry', 'RB', 1, 29.5)
('Davante Adams', 'WR', 1, 23.5)
('Derrick Henry', 'RB', 2, 41.0)
('Davante Adams', 'WR', 2, 39.0)
('Josh Allen', 'QB', 2, 14.0)
